In [1]:
# -------------------------
# Dependencies
# -------------------------

!pip install torch torchvision facenet-pytorch efficientnet-pytorch opencv-python pillow

# -------------------------
# Imports
# -------------------------
import os
import cv2
import torch
import numpy as np
import torch.nn as nn
from torchvision import transforms
from facenet_pytorch import MTCNN
from efficientnet_pytorch import EfficientNet

# Detect environment
try:
    from google.colab import files
    ENV = "colab"
except ImportError:
    ENV = "jupyter"
    from tkinter import Tk
    from tkinter.filedialog import askopenfilename

print(f"Running in {ENV.upper()} environment")

# -------------------------
# Device setup
# -------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# -------------------------
# Load MTCNN face detector
# -------------------------
print(" Loading MTCNN face detector...")
mtcnn = MTCNN(keep_all=True, device=DEVICE)

# -------------------------
# Build EfficientNet-B0 model
# -------------------------
print(" Building EfficientNet-B0...")
model = EfficientNet.from_name("efficientnet-b0")
model._fc = nn.Linear(model._fc.in_features, 1)

# Optional: Custom weights path
WEIGHT_PATH = "efficientnet_b0_epoch_15_loss_0.158.pth"
if os.path.exists(WEIGHT_PATH):
    print(f" Loading custom weights from {WEIGHT_PATH}")
    state_dict = torch.load(WEIGHT_PATH, map_location=DEVICE)
    model.load_state_dict(state_dict)
else:
    print("⚠ No custom weights found. Using ImageNet-pretrained EfficientNet instead.")
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(model._fc.in_features, 1)

model = model.to(DEVICE)
model.eval()

# -------------------------
# Image preprocessing
# -------------------------
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# -------------------------
# Prediction function
# -------------------------
def predict_on_video(video_path, max_frames=50):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_process = min(max_frames, frame_count)

    predictions = []
    idx = 0

    while idx < frames_to_process:
        ret, frame = cap.read()
        if not ret:
            break
        idx += 1

        # Detect faces
        boxes, _ = mtcnn.detect(frame)
        if boxes is None:
            continue

        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]
            if face.size == 0:
                continue

            face_tensor = transform(face).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                output = model(face_tensor)
                pred = torch.sigmoid(output).item()
                predictions.append(pred)

    cap.release()

    if not predictions:
        return "⚠ No face detected in video."

    avg_score = np.mean(predictions)
    return f"Video is {avg_score*100:.2f}% likely to be a FAKE."

# -------------------------
# Video selection (Colab vs Jupyter)
# -------------------------
if ENV == "colab":
    print(" Upload your video (MP4 format)")
    uploaded = files.upload()
    video_path = f"/content/{list(uploaded.keys())[0]}"
    print(f" Video uploaded: {video_path}")

else:
    # Jupyter / Mac
    Tk().withdraw()  # hide main window
    video_path = askopenfilename(title="Select video file", filetypes=[("MP4 files", "*.mp4")])
    if not video_path:
        raise FileNotFoundError("No video selected.")
    print(f" Video selected: {video_path}")

# -------------------------
# Run inference
# -------------------------
print("🔹 Running deepfake inference...")
try:
    result = predict_on_video(video_path)
    print(" Prediction Result:")
    print(result)
except Exception as e:
    print(" Error during inference:", e)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of facenet-pytorch to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.6 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=a10b093da935540a742f83446d70d0062c62e176437e6a06c3e8f8134b9ed726
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet-pytorch
Running in COLAB environment
Using device: cuda
 Loading MTCNN face detector...
 Building EfficientNet-B0...
⚠ No custom weights found. Using ImageNet-pretrained EfficientNet instead.
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub

100%|██████████| 20.4M/20.4M [00:00<00:00, 196MB/s]

Loaded pretrained weights for efficientnet-b0
 Upload your video (MP4 format)


KeyboardInterrupt: 